In [19]:
import os
import cv2
import numpy as np

# Function to create directories if they don't exist
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to normalize and apply colormap for NDVI
def apply_ndvi_colormap(ndvi_image):
    # Normalize NDVI values to [0, 255] for colormap application
    normalized_ndvi = cv2.normalize(ndvi_image, None, 0, 255, cv2.NORM_MINMAX)
    # Apply a colormap (COLORMAP_JET for red-to-green)
    color_mapped_ndvi = cv2.applyColorMap(normalized_ndvi.astype(np.uint8), cv2.COLORMAP_JET)
    return color_mapped_ndvi

# Function to slice images into smaller patches
def slice_images(image_pairs, output_dir, patch_size=64):
    # Ensure the output directories exist
    normal_dir = os.path.join(output_dir, "normal")
    ndvi_dir = os.path.join(output_dir, "ndvi")
    ensure_dir(normal_dir)
    ensure_dir(ndvi_dir)

    patch_id = 0  # Counter for naming patches

    for true_color_path, ndvi_path in image_pairs:
        # Load the images
        img = cv2.imread(true_color_path)  # Load true-color image
        ndvi = cv2.imread(ndvi_path, cv2.IMREAD_GRAYSCALE)  # Load NDVI image as grayscale

        if img is None or ndvi is None:
            print(f"Error loading images: {true_color_path}, {ndvi_path}. Skipping...")
            continue

        # Apply the colormap to the NDVI image
        ndvi_colormap = apply_ndvi_colormap(ndvi)

        h, w, _ = img.shape
        ph, pw = patch_size, patch_size

        # Slice images into patches
        for i in range(0, h - ph + 1, ph):
            for j in range(0, w - pw + 1, pw):
                # Extract patches
                true_color_patch = img[i:i + ph, j:j + pw]
                ndvi_patch = ndvi_colormap[i:i + ph, j:j + pw]

                # Save patches
                true_color_patch_path = os.path.join(normal_dir, f"patch_{patch_id}.png")
                ndvi_patch_path = os.path.join(ndvi_dir, f"patch_{patch_id}.png")

                cv2.imwrite(true_color_patch_path, true_color_patch)
                cv2.imwrite(ndvi_patch_path, ndvi_patch)

                patch_id += 1

    print(f"Saved {patch_id} patches to {output_dir}")

# List of image pairs (true color, NDVI)
image_pairs = [
    ("../color/2_True_color.jpg", "../ndvi/2_NDVI.jpg"),
    ("../color/3_True_color.jpg", "../ndvi/3_NDVI.jpg"),
]

# Output directory
output_directory = "combined_patches"

# Slice images
slice_images(image_pairs, output_directory, patch_size=64)

Saved 722 patches to combined_patches


In [20]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_unet(input_shape=(64, 64, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)  # Single NDVI output channel

    model = Model(inputs, outputs)
    return model

In [21]:
import os
import numpy as np
from tensorflow.keras.utils import Sequence
import cv2

# Data generator to load patches on-the-fly
class PatchGenerator(Sequence):
    def __init__(self, normal_dir, ndvi_dir, batch_size):
        self.normal_paths = sorted([os.path.join(normal_dir, f) for f in os.listdir(normal_dir)])
        self.ndvi_paths = sorted([os.path.join(ndvi_dir, f) for f in os.listdir(ndvi_dir)])
        self.batch_size = batch_size

    def __len__(self):
        return len(self.normal_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_normal = self.normal_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_ndvi = self.ndvi_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        X = []
        Y = []

        for normal_path, ndvi_path in zip(batch_normal, batch_ndvi):
            normal_img = cv2.imread(normal_path) / 255.0  # Normalize to [0, 1]
            ndvi_img = cv2.imread(ndvi_path)[:, :, 1] / 255.0  # Extract green channel, normalize

            X.append(normal_img)
            Y.append(np.expand_dims(ndvi_img, axis=-1))  # Add channel dimension

        return np.array(X), np.array(Y)

# Paths to the patch directories
normal_dir = "combined_patches/normal"
ndvi_dir = "combined_patches/ndvi"

# Create data generator
batch_size = 16
train_gen = PatchGenerator(normal_dir, ndvi_dir, batch_size)


In [22]:
# Build the model
model = build_unet(input_shape=(64, 64, 3))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(train_gen, epochs=10)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 241ms/step - loss: 0.1785 - mae: 0.3929
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 14s 312ms/step - loss: 0.1206 - mae: 0.2953
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 249ms/step - loss: 0.1155 - mae: 0.2659
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 14s 299ms/step - loss: 0.1055 - mae: 0.2725
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 235ms/step - loss: 0.0937 - mae: 0.2465
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 292ms/step - loss: 0.0698 - mae: 0.1950
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 297ms/step - loss: 0.1347 - mae: 0.2512
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 247ms/step - loss: 0.0722 - mae: 0.2118
Epoch 9/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 14s 300ms/step - loss: 0.0876 - mae: 0.2124
Epoch 10/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 239ms/step - loss: 0.0738 - mae: 0.2156


In [25]:
def predict_large_image(model, image_path, patch_size=64, stride=32):
    img = cv2.imread(image_path) / 255.0  # Normalize input image
    h, w, _ = img.shape

    # Pad image to make it divisible by patch_size
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    img_padded = np.pad(img, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')

    output = np.zeros((h + pad_h, w + pad_w, 1))

    # Sliding window over the image
    for i in range(0, img_padded.shape[0] - patch_size + 1, stride):
        for j in range(0, img_padded.shape[1] - patch_size + 1, stride):
            patch = img_padded[i:i + patch_size, j:j + patch_size]
            pred_patch = model.predict(np.expand_dims(patch, axis=0))[0]
            output[i:i + patch_size, j:j + patch_size] += pred_patch

    # Crop back to original size
    return output[:h, :w]

# Predict on a large image
large_image_path = "../test/img.png"
ndvi_prediction = predict_large_image(model, large_image_path)

# Apply a red-to-green colormap and save the NDVI prediction
def save_ndvi_with_colormap(ndvi_prediction, output_path):
    # Normalize NDVI values to the range [0, 255]
    normalized_ndvi = cv2.normalize(ndvi_prediction, None, 0, 255, cv2.NORM_MINMAX)
    # Apply the red-to-green colormap (COLORMAP_JET)
    ndvi_colormap = cv2.applyColorMap(normalized_ndvi.astype(np.uint8), cv2.COLORMAP_JET)
    # Save the colored NDVI image
    cv2.imwrite(output_path, ndvi_colormap)

# Save the NDVI prediction with the colormap
output_path = "../test/img_NDVI_PRED.jpg"
save_ndvi_with_colormap(ndvi_prediction, output_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━